In [1]:
#Extract Training data
raw_numeric = sc.textFile("train_numeric.csv.gz")

#Remove header
header = raw_numeric.first()
raw_numeric_noheader = raw_numeric.filter(lambda line:line != header)

#Count the dataset
numeric_count = raw_numeric_noheader.count()
print "Total Numeric Count is {} ".format(numeric_count)

## Sample Training and trainign data from the cleaned raw RDD!
raw_training_sample = raw_numeric_noheader.sample(False, 0.7, 1234)
raw_test_sample = raw_numeric_noheader.sample(False, 0.3, 12345)

training_count = raw_training_sample.count() 
test_count = raw_test_sample.count()
print"Training data count is  {} and Test data count is {}  ".format(training_count,test_count)


Total Numeric Count is 1183747 
Training data count is  828549 and Test data count is 354686  


In [2]:
#Convert raw data to csv
csv_training_data = raw_training_sample.map(lambda x: x.split(","))
csv_test_data = raw_test_sample.map(lambda x: x.split(","))
csv_total_data = raw_numeric_noheader.map(lambda x: x.split(","))

# Analysze the defect part with in total training set 
id_response_numeric_key_value =  csv_total_data.map(lambda x: ( x[-1], 1 )) 
defect_numeric_key_value = id_response_numeric_key_value.filter(lambda y: y[0]=="1" )
defect_counts_by_key = defect_numeric_key_value.countByKey()
defect_percent = 100*float(defect_counts_by_key.values()[0])/float(numeric_count)
print"{} percent of products are found to be defective".format(round(defect_percent,3))

# Analysze the defect part with in split training data sets 
id_response_train_key_value =  csv_training_data.map(lambda x: ( x[-1], 1 )) 
defect_train_key_value = id_response_train_key_value.filter(lambda y: y[0]=="1" )
defect_counts_by_key = defect_train_key_value.countByKey()
defect_percent = 100*float(defect_counts_by_key.values()[0])/float(training_count)
print"{} percent of products  found to be defective in training ".format(round(defect_percent,3))

# Analysze the defect part with in split test data sets 
id_response_test_key_value =  csv_test_data.map(lambda x: ( x[-1], 1 )) 
defect_test_key_value = id_response_test_key_value.filter(lambda y: y[0]=="1" )
defect_counts_by_key = defect_test_key_value.countByKey()
defect_percent = 100*float(defect_counts_by_key.values()[0])/float(test_count)
print"{} percent of products are found to be defective in test set ".format(round(defect_percent,3))

0.581 percent of products are found to be defective
0.585 percent of products  found to be defective in training 
0.58 percent of products are found to be defective in test set 


In [25]:
#A function to Create Labelpoint for training and validation
from pyspark.mllib.regression import LabeledPoint
from numpy import array
from numpy import nan

def empty_element_to_dense_lv(line):
    nline = line[1:-1]
    for i,item in enumerate(nline):
        if item == '':
            nline[i] = nan
    return LabeledPoint(float(line[-1]), array([float(x) for x in nline]))

## Create data for Supervised learning 
training_data = csv_training_data.map(empty_element_to_dense_lv)
test_data = csv_test_data.map(empty_element_to_dense_lv)

#Train the model 
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from time import time

# Build the model
t0 = time()
tree_model = DecisionTree.trainClassifier(training_data,2,{},'gini',16,120)
tt = time() - t0


print "Decision tree Classifier trained in {} seconds".format(round(tt,3))

Decision tree Classifier trained in 2555.466 seconds


In [26]:
from numpy import sqrt
## Predict 
predictions = tree_model.predict(test_data.map(lambda p: p.features))
labels_and_preds = test_data.map(lambda p: p.label).zip(predictions)

#Accuracy checks
t0 = time()
test_accuracy = labels_and_preds.filter(lambda (v, p): v == p).count() / float(test_data.count())
tt = time() - t0

print "Prediction made in {} seconds. Test accuracy is {}".format(round(tt,3), round(test_accuracy,4))

#Compute MCC
t0 = time()

TP = labels_and_preds.filter(lambda (v, p): int(v) == 1 and  int(p) == 1 ).count()
FP = labels_and_preds.filter(lambda (v, p): int(v) == 0 and  int(p)  == 1 ).count()
FN = labels_and_preds.filter(lambda (v, p): int(v) == 1 and  int(p) == 0 ).count()
TN = labels_and_preds.filter(lambda (v, p): int(v) == 0 and  int(p) == 0 ).count()
MCC = float(TP*TN - FP*FN)/float(sqrt( (TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)   ) )
tt = time() - t0
print "MCC is caluclated in  {} seconds. The MCC for Decision tree model is : {} ".format(round(tt,3), round(MCC,4))


Prediction made in 248.719 seconds. Test accuracy is 0.9942
MCC is caluclated in  527.847 seconds. The MCC for Decision tree model is : 0.1197 


In [27]:
tree_model.save(sc,"Decision_tree_best_nan")